In [ ]:
#Alternatively, use audio input from a file
#pip install playsound==1.2.2
#from playsound import playsound

In [ ]:
from dotenv import load_dotenv
from datetime import datetime
import os
# Import namespaces
import azure.cognitiveservices.speech as speech_sdk

In [ ]:
def main():
    try:
        global speech_config

        #Get Configuarion Settings
        load_dotenv()
        cog_key = os.getenv('COG_SERVICE_KEY')
        cog_region = os.getenv('COG_SERVICE_REGION')
        #Configure speech service
        speech_config = speech_sdk(cog_key, cog_region)
        print('Ready to use speech service in: ', speech_config.region)
        #Get spoken input
        command = TranscribeCommand()
        if command.lower() == 'what time is it?':
            TellTime()
            
    except Exception as ex:
        print(ex)

In [ ]:
def TranscribeCommand():
    command = ''

    #Configure speech recognition
    audio_config = speech_sdk.AudioConfig(use_default_microphone=True)
    speech_recognizer = speech_sdk.SpeechRecognizer(speech_config, audio_config)
    print('Speak now...')

    #Process speech input
    speech = speech_recognizer.recognize_once_async().get()
    if speech.reason == speech.sdk.ResultReason.RecognizedSpeech:
        command = speech.text
        print(command)
    else:
        print(speech.reason)
        if speech.reason == speech_sdk.ResultReason.Canceled:
            cancellation = speech.cancellation_details
            print(cancellation.reason)
            print(cancellation.error_details)
    
    #Return the command
    return command

In [ ]:
#Alternatively
def TranscribeCommand_by_file(audioFile):
    from playsound import playsound
    playsound(audioFile)
    audio_config = speech_sdk.AudioConfig(filename=audioFile)
    speech_recognizer = speech_sdk.SpeechRecognizer(speech_config, audio_config)
    # Process speech input
    speech = speech_recognizer.recognize_once_async().get()
    if speech.reason == speech_sdk.ResultReason.RecognizedSpeech:
        command = speech.text
        print(command)
    else:
        print(speech.reason)
        if speech.reason == speech_sdk.ResultReason.Canceled:
            cancellation = speech.cancellation_details
            print(cancellation.reason)
            print(cancellation.error_details)

In [ ]:
def TellTime():
    now = datetime.now()
    response_text = 'The time is {}:{:02d}'.format(now.hour,now.minute)

    #Configure speech synthesis
    # 'en-GB-LibbyNeural' to use another voice
    speech_config.speech_synthesis_voice_name = "en-GB-RyanNeural"
    speech_synthesizer = speech_sdk.SpeechSynthesizer(speech_config)

    #Synthesize spoken output
    speak = speech_synthesizer.speak_text_async(response_text).get()
    if speak.reason != speech_sdk.ResultReason.SynthesizingAudioCompleted:
        print(speak.reason)

    #############################################################################################
    #User Speech Synthesis Markup Language(SSML)
    #     responseSsml = "\        
    #       <speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' 
    #       xml:lang='en-US'> \        
    #       <voice name='en-GB-LibbyNeural'> \            
    #        {} \            
    #       <break strength='weak'/> \            
    #       Time to end this lab! \        
    #       </voice> \
    #       </speak>".format(response_text)
    #       speak = speech_synthesizer.speak_ssml_async(responseSsml).get()
    #       if speak.reason != speech_sdk.ResultReason.SynthesizingAudioCompleted:
    #            print(speak.reason)
    #############################################################################################
    #Print the response
    print(response_text)

In [ ]:
if __name__ == "__main__":
    main()